In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [4]:
df = pd.read_csv('COVID-19_and_Price_dataset.csv')

print(df.shape)  
df.tail()

(115, 834)


,Date,Afghanistan_total_cases,Afghanistan_new_cases,Afghanistan_total_deaths,Afghanistan_new_deaths,Albania_total_cases,Albania_new_cases,Albania_total_deaths,Albania_new_deaths,Algeria_total_cases,...,Zambia_new_deaths,Zimbabwe_total_cases,Zimbabwe_new_cases,Zimbabwe_total_deaths,Zimbabwe_new_deaths,World_total_cases,World_new_cases,World_total_deaths,World_new_deaths,Price
110,2020-06-09,20917,575,369,12,1263,17,34,0,10265,...,3,287,5,4,0,7084384,105851,408161,3230,27.558533
111,2020-06-10,21459,542,384,15,1299,36,34,0,10382,...,0,314,27,4,0,7209978,125594,413090,4929,27.400933
112,2020-06-11,22143,684,405,21,1341,42,34,0,10484,...,0,320,6,4,0,7343440,133462,418329,5239,27.222933
113,2020-06-12,22890,747,426,21,1385,44,35,1,10589,...,0,332,12,4,0,7481017,137577,423091,4762,27.057200
114,2020-06-15,24766,664,471,20,1521,57,36,0,10919,...,1,383,27,4,0,7881616,122105,435154,3129,26.922533


In [5]:
df = df.dropna(axis=1)

In [6]:
df.tail()

,Date,Afghanistan_total_cases,Afghanistan_new_cases,Afghanistan_total_deaths,Afghanistan_new_deaths,Albania_total_cases,Albania_new_cases,Albania_total_deaths,Albania_new_deaths,Algeria_total_cases,...,Zambia_new_deaths,Zimbabwe_total_cases,Zimbabwe_new_cases,Zimbabwe_total_deaths,Zimbabwe_new_deaths,World_total_cases,World_new_cases,World_total_deaths,World_new_deaths,Price
110,2020-06-09,20917,575,369,12,1263,17,34,0,10265,...,3,287,5,4,0,7084384,105851,408161,3230,27.558533
111,2020-06-10,21459,542,384,15,1299,36,34,0,10382,...,0,314,27,4,0,7209978,125594,413090,4929,27.400933
112,2020-06-11,22143,684,405,21,1341,42,34,0,10484,...,0,320,6,4,0,7343440,133462,418329,5239,27.222933
113,2020-06-12,22890,747,426,21,1385,44,35,1,10589,...,0,332,12,4,0,7481017,137577,423091,4762,27.057200
114,2020-06-15,24766,664,471,20,1521,57,36,0,10919,...,1,383,27,4,0,7881616,122105,435154,3129,26.922533


In [7]:
df.shape


(115, 708)

In [8]:
cor = df.corr()
#Correlation with output variable
cor_target = abs(cor["Price"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.98]
relevant_features

Albania_total_cases                 0.985852
Algeria_total_deaths                0.980385
Andorra_total_deaths                0.984346
Australia_total_deaths              0.987319
Austria_total_deaths                0.985505
Bahamas_total_cases                 0.988639
Belgium_total_cases                 0.990456
Bermuda_total_cases                 0.987915
BosniaandHerzegovina_total_cases    0.987825
BritishVirginIslands_total_cases    0.988040
BurkinaFaso_total_cases             0.991982
BurkinaFaso_total_deaths            0.987984
Cuba_total_cases                    0.981009
Cyprus_total_cases                  0.983114
Cyprus_total_deaths                 0.983710
CzechRepublic_total_cases           0.989432
CzechRepublic_total_deaths          0.983694
Denmark_total_cases                 0.995615
Denmark_total_deaths                0.988755
France_total_cases                  0.988345
France_total_deaths                 0.983850
Georgia_total_cases                 0.987645
Germany_to

In [9]:
df = df[[ 'BritishVirginIslands_total_cases' , 'Cyprus_total_deaths',
        'Grenada_total_cases', 'Guyana_total_deaths'   , 'Niger_total_cases' ,
         'Singapore_total_deaths', 'SriLanka_total_deaths','Price'] ]

In [10]:
df.shape

(115, 8)

#### to run a Granger Causality test, the time series' you are using must be stationary

In [11]:

from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    

    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df, variables = df.columns)           

,BritishVirginIslands_total_cases_x,Cyprus_total_deaths_x,Grenada_total_cases_x,Guyana_total_deaths_x,Niger_total_cases_x,Singapore_total_deaths_x,SriLanka_total_deaths_x,Price_x
BritishVirginIslands_total_cases_y,1.0000,0.0,0.0004,0.0000,0.0001,0.0000,0.0222,0.0000
Cyprus_total_deaths_y,0.0000,1.0,0.0000,0.0000,0.0136,0.0000,0.2457,0.0000
Grenada_total_cases_y,0.4568,0.0,1.0000,0.0094,0.0264,0.0018,0.1239,0.0000
Guyana_total_deaths_y,0.0000,0.0,0.0000,1.0000,0.0000,0.0016,0.0000,0.0000
Niger_total_cases_y,0.0000,0.0,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
Singapore_total_deaths_y,0.0000,0.0,0.0000,0.0000,0.0025,1.0000,0.0001,0.0000
SriLanka_total_deaths_y,0.0000,0.0,0.0000,0.0000,0.0027,0.0002,1.0000,0.0008
Price_y,0.0000,0.0,0.1141,0.0000,0.0000,0.0000,0.0000,1.0000


In [12]:
nobs = 30
df_train, df_test = df[0:-nobs], df[-nobs:]

# Check size
print(df_train.shape)  # (85, 8)
print(df_test.shape)  # (30, 8)

(85, 8)
(30, 8)


In [13]:
test =df_test[['Price']]

In [14]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [15]:
df_train.head(1)

,BritishVirginIslands_total_cases,Cyprus_total_deaths,Grenada_total_cases,Guyana_total_deaths,Niger_total_cases,Singapore_total_deaths,SriLanka_total_deaths,Price
0,0,0,0,0,0,0,0,56.9496


In [16]:
# ADF Test on each column
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BritishVirginIslands_total_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 1.4777
 No. Lags Chosen       = 0
 Critical value 1%     = -3.511
 Critical value 5%     = -2.897
 Critical value 10%    = -2.585
 => P-Value = 0.9974. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "Cyprus_total_deaths" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 0.2014
 No. Lags Chosen       = 3
 Critical value 1%     = -3.514
 Critical value 5%     = -2.898
 Critical value 10%    = -2.586
 => P-Value = 0.9723. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "Grenada_total_cases" 
    --------------------------

In [17]:
# 1st difference
df_differenced = df_train.diff().dropna()

In [18]:
# ADF Test on each column of 1st Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BritishVirginIslands_total_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.927
 No. Lags Chosen       = 1
 Critical value 1%     = -3.513
 Critical value 5%     = -2.897
 Critical value 10%    = -2.586
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Cyprus_total_deaths" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.1011
 No. Lags Chosen       = 2
 Critical value 1%     = -3.514
 Critical value 5%     = -2.898
 Critical value 10%    = -2.586
 => P-Value = 0.0265. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Grenada_total_cases" 
    -----------------------------------------------
 Null Hypothesis: Data h

In [19]:
# Second Differencing
df_differenced = df_differenced.diff().dropna()

In [20]:
# ADF Test on each column of 2nd Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BritishVirginIslands_total_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.6173
 No. Lags Chosen       = 3
 Critical value 1%     = -3.516
 Critical value 5%     = -2.899
 Critical value 10%    = -2.587
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Cyprus_total_deaths" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -5.6553
 No. Lags Chosen       = 7
 Critical value 1%     = -3.521
 Critical value 5%     = -2.901
 Critical value 10%    = -2.588
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Grenada_total_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has

In [21]:
# third Differencing
df_differenced = df_differenced.diff().dropna()

In [22]:
# ADF Test on each column of 3rd Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "BritishVirginIslands_total_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -5.9622
 No. Lags Chosen       = 7
 Critical value 1%     = -3.522
 Critical value 5%     = -2.901
 Critical value 10%    = -2.588
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Cyprus_total_deaths" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.9536
 No. Lags Chosen       = 12
 Critical value 1%     = -3.529
 Critical value 5%     = -2.904
 Critical value 10%    = -2.59
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Grenada_total_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has

In [23]:
model = VAR(df_differenced)
for i in [1,2,3,4,5]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  -6.013451565056346
BIC :  -3.885052316458622
FPE :  0.002463724789011075
HQIC:  -5.159509896191297 

Lag Order = 2
AIC :  -12.264439337571751
BIC :  -8.214994058626154
FPE :  4.97129931378965e-06
HQIC:  -10.640900851764274 

Lag Order = 3
AIC :  -20.927140326811738
BIC :  -14.928538168667378
FPE :  9.763727900388386e-10
HQIC:  -18.523916774534058 

Lag Order = 4
AIC :  -76.66485407551855
BIC :  -68.68830112364608
FPE :  7.978496693040108e-34
HQIC:  -73.47169341302866 

Lag Order = 5
AIC :  -384.5629752255239
BIC :  -374.5789729090563
FPE :  2.5768823404929766e-167
HQIC:  -380.56946200812416 



C:\Users\ramsu\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [45]:
x = model.select_order(maxlags=5)
x.summary()

,AIC,BIC,FPE,HQIC
0,2.405,2.648,11.08,2.502
1,-5.559,-3.368,0.003885,-4.683
2,-11.91,-7.772,7.123e-06,-10.26
3,-20.79,-14.70,1.137e-09,-18.35
4,-76.53,-68.50,9.296e-34,-73.32
5,-381.7*,-371.7*,4.736e-166*,-377.7*


In [24]:
model_fitted = model.fit()
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 15, Sep, 2022
Time:                     12:21:32
--------------------------------------------------------------------
No. of Equations:         8.00000    BIC:                   -3.88505
Nobs:                     81.0000    HQIC:                  -5.15951
Log likelihood:          -603.927    FPE:                 0.00246372
AIC:                     -6.01345    Det(Omega_mle):      0.00106055
--------------------------------------------------------------------
Results for equation BritishVirginIslands_total_cases
                                         coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------------------------
const                                       0.003155         0.037985            0.083           0.934
L1.BritishVirginIslands_total_cases        -0.79

In [25]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
forecast_input

1


array([[ 0.        ,  0.        , -4.        ,  1.        ,  6.        ,
         1.        ,  0.        , -0.06293333]])

In [26]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns + '_2d')
df_forecast

,BritishVirginIslands_total_cases_2d,Cyprus_total_deaths_2d,Grenada_total_cases_2d,Guyana_total_deaths_2d,Niger_total_cases_2d,Singapore_total_deaths_2d,SriLanka_total_deaths_2d,Price_2d
85,0.453107,-2.580632,1.125532,-0.854653,58.781384,-1.813522,-0.659684,0.189938
86,0.139316,2.316463,2.147137,0.843679,-51.902286,0.770230,0.524222,-0.180316
87,-0.842913,-0.388850,-3.402973,-0.621069,20.747801,0.743908,-0.148914,0.114180
88,1.061644,-1.488495,2.229328,0.252565,1.040835,-1.563726,-0.200226,-0.047914
89,-0.690969,2.058679,0.002329,0.087824,-3.839046,1.340445,0.401443,0.020303
90,0.082434,-1.394825,-1.700331,-0.154621,0.910159,-0.479310,-0.454398,-0.018426
91,0.413642,0.172959,2.008189,0.018213,-0.051774,-0.350453,0.326115,0.030342
92,-0.570051,0.685248,-1.231294,0.197173,3.992640,0.700501,-0.081166,-0.030680
93,0.454357,-0.894360,0.149213,-0.254354,-4.554232,-0.539607,-0.202059,0.023794
94,-0.202296,0.510968,0.489912,0.158294,0.427363,0.135068,0.367835,-0.007060


In [27]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [28]:
df_results = invert_transformation(df_train, df_forecast, second_diff=True)        


In [29]:
predict = df_results['Price_forecast']

In [30]:
len(predict)

30

In [32]:
predict

85     34.783272
86     34.277961
87     33.886830
88     33.447785
89     33.029043
90     32.591875
91     32.185048
92     31.747542
93     31.333829
94     30.913057
95     30.489426
96     30.075997
97     29.655334
98     29.240603
99     28.825052
100    28.411068
101    27.997875
102    27.586087
103    27.175126
104    26.765273
105    26.356506
106    25.948914
107    25.542122
108    25.136654
109    24.732204
110    24.328592
111    23.926426
112    23.525009
113    23.124758
114    22.725693
Name: Price_forecast, dtype: float64

In [31]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt( mean_squared_error(test,predict))

1.986291591844842